In [1]:
import pandas as pd
import numpy as np
ped = pd.read_csv('gmm.csv')
# ped.loc[ped['Father'] == ped.loc[0,'Father'],'Father'] = 0
# ped.loc[ped['Mother'] == ped.loc[0,'Mother'],'Mother'] = 0
ped['Sex'] = ped['Sex'].fillna('Unknown')
ped.fillna(value=0,inplace=True)
ped

,ID,Father,Mother,Sex,Object
0,1,0.0,0.0,Unknown,0.0
1,2,0.0,0.0,Unknown,0.0
2,3,0.0,0.0,1.0,850.0
3,4,1.0,0.0,2.0,803.0
4,5,1.0,2.0,1.0,830.0
5,6,1.0,2.0,2.0,790.0
6,7,3.0,4.0,1.0,820.0
7,8,3.0,6.0,1.0,835.0
8,9,3.0,6.0,2.0,779.0


In [2]:
id_to_idx = {id: idx for idx, id in enumerate(ped["ID"])}
n = len(ped)
A = np.eye(n)
for i in range(3,9):
    Father = int(ped[ped['ID']== i+1]['Father'].values[0])
    Mother = int(ped[ped['ID']== i+1]['Mother'].values[0])
    for b in range(0,i):
        if Mother==0:
            A[b][i] = (A[b][Father-1] + 0)/2
            A[i][b] = (A[b][Father-1] + 0)/2
        else:
            A[b][i] = (A[b][Father-1] + A[b][Mother-1])/2
            A[i][b] = (A[b][Father-1] + A[b][Mother-1])/2
A

array([[1.    , 0.    , 0.    , 0.5   , 0.5   , 0.5   , 0.25  , 0.25  ,
        0.25  ],
       [0.    , 1.    , 0.    , 0.    , 0.5   , 0.5   , 0.    , 0.25  ,
        0.25  ],
       [0.    , 0.    , 1.    , 0.    , 0.    , 0.    , 0.5   , 0.5   ,
        0.5   ],
       [0.5   , 0.    , 0.    , 1.    , 0.25  , 0.25  , 0.5   , 0.125 ,
        0.125 ],
       [0.5   , 0.5   , 0.    , 0.25  , 1.    , 0.5   , 0.125 , 0.25  ,
        0.25  ],
       [0.5   , 0.5   , 0.    , 0.25  , 0.5   , 1.    , 0.125 , 0.5   ,
        0.5   ],
       [0.25  , 0.    , 0.5   , 0.5   , 0.125 , 0.125 , 1.    , 0.3125,
        0.3125],
       [0.25  , 0.25  , 0.5   , 0.125 , 0.25  , 0.5   , 0.3125, 1.    ,
        0.5   ],
       [0.25  , 0.25  , 0.5   , 0.125 , 0.25  , 0.5   , 0.3125, 0.5   ,
        1.    ]])

In [17]:
inv_A = np.linalg.inv(A)

import numpy as np
from fractions import Fraction

def print_matrix_as_fractions(matrix):
    """将矩阵的每个元素转换为分数形式并打印"""
    rows, cols = matrix.shape
    for i in range(rows):
        row = [f"{Fraction(x).limit_denominator()}" for x in matrix[i]]
        print(" ".join(row))
print("表格法计算亲缘关系矩阵为：")
print_matrix_as_fractions(inv_A)


表格法计算亲缘关系矩阵为：
7/3 1 0 -2/3 -1 -1 0 0 0
1 2 0 0 -1 -1 0 0 0
0 0 5/2 1/2 0 1 -1 -1 -1
-2/3 0 1/2 11/6 0 0 -1 0 0
-1 -1 0 0 2 0 0 0 0
-1 -1 1 0 0 3 0 -1 -1
0 0 -1 -1 0 0 2 0 0
0 0 -1 0 0 -1 0 2 0
0 0 -1 0 0 -1 0 0 2


In [15]:
h2 = 0.4
k = (1-h2)/h2
ZZ_1 = np.eye(9)*np.eye(9).T
ZZ_1[0][0] = ZZ_1[1][1] =0
kA_1 = k*inv_A
d = ZZ_1 + kA_1
df6 = ped.copy()
X = pd.get_dummies(ped.copy(),columns=['Sex']).loc[:,'Sex_1.0':'Sex_2.0'].to_numpy().astype(int)
X_Z = np.dot(X.T,np.eye(9))
Z_X = np.dot(np.eye(9).T,X)
X_X =np.dot(X.T,X)
MME = np.block(
    [[X_X,X_Z],
     [Z_X,d]]
)
h1 = ped[ped['Sex'] == 1]['Object'].sum()
h2 = ped[ped['Sex'] == 2]['Object'].sum()
a1_9 = ped['Object'].to_numpy()[:,None]
C = np.block(
    [[h1],
     [h2],
     [a1_9]]
)
inv_MME = np.linalg.inv(MME)
S = np.dot(inv_MME,C)
np.set_printoptions(precision=4, suppress=True)
S = f"左上角：\n{X_X}\n 右上角：\n{X_Z}\n 左下角：\n{Z_X}\n 右下角：\n{d}\n 总矩阵：\n{MME}"
print("根据矩阵方程计算：")
print(f"其中X：\n{X}\n Z：\n{np.eye(9)}\n k：{k}")
print(S)

根据矩阵方程计算：
其中X：
[[0 0]
 [0 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]]
 Z：
[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
 k：1.4999999999999998
左上角：
[[4 0]
 [0 3]]
 右上角：
[[0. 0. 1. 0. 1. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 1.]]
 左下角：
[[0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]
 右下角：
[[ 3.5   1.5   0.   -1.   -1.5  -1.5   0.    0.    0.  ]
 [ 1.5   3.    0.    0.   -1.5  -1.5   0.    0.    0.  ]
 [ 0.    0.    4.75  0.75  0.    1.5  -1.5  -1.5  -1.5 ]
 [-1.    0.    0.75  3.75  0.    0.   -1.5   0.    0.  ]
 [-1.5  -1.5   0.    0.    4.    0.    0.    0.    0.  ]
 [-1.5  -1.5   1.5   0.    0.    5.5   0.   -1.5  -1.5 ]
 [ 0.    0.   -1.5  -1.5   0.    0.    4.    0.    0.  ]
 [ 0.    0.   -1.5   0.    0.   -1.5   0.   

In [5]:
S_x = np.vsplit(S, [2, ])
print(f'Y = X{S_x[0]} + Z{S_x[1]} + e')
# print(S_x)

Y = X[[834.4889]
 [792.4892]] + Z[[-0.9935]
 [-2.1009]
 [ 1.9869]
 [ 1.1644]
 [-2.2826]
 [-2.9126]
 [-2.4405]
 [-0.2193]
 [-3.7194]] + e


In [6]:
# 线性混合模型
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [7]:
# 生成模拟数据
np.random.seed(42)
n = 100  # 学生数
schools = np.repeat(["A", "B", "C", "D"], n // 4)  # 4个学校
hours_studied = np.random.normal(5, 2, n)  # 学习时间（固定效应）
school_effect = np.random.normal(0, 3, size=4)  # 学校随机效应
score = 50 + 2 * hours_studied + pd.Series(schools).astype("category").cat.codes.to_numpy() + np.random.normal(0, 1, n)

# 创建DataFrame
data = pd.DataFrame({
    "school": schools,
    "hours": hours_studied,
    "score": score
})
data

,school,hours,score
0,A,5.993428,61.825571
1,A,4.723471,59.850994
2,A,6.295377,64.476940
3,A,8.046060,66.266697
4,A,4.531693,59.320937
...,...,...,...
95,D,2.072970,56.002970
96,D,5.592241,64.542268
97,D,5.522111,64.605006
98,D,5.010227,64.103505


In [8]:
model = sm.MixedLM.from_formula(
    "score ~ hours",   # 固定效应：score由hours解释
    data=data,
    groups=data["school"],  # 按school分组
    re_formula = "~0 + hours"          # 随机效应为截距（默认）
)
result = model.fit()  # 拟合模型

In [9]:
print(result.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: score    
No. Observations: 100     Method:             REML     
No. Groups:       4       Scale:              1.0211   
Min. group size:  25      Log-Likelihood:     -150.6861
Max. group size:  25      Converged:          Yes      
Mean group size:  25.0                                 
-------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept   51.074    0.289 176.647 0.000 50.507 51.641
hours        2.100    0.132  15.935 0.000  1.841  2.358
hours Var    0.057    0.048                            

